In [10]:
pip install tableauhyperapi


   ---------------------------------------- 0.0/53.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/53.8 MB 1.5 MB/s eta 0:00:36
   ---------------------------------------- 0.3/53.8 MB 3.5 MB/s eta 0:00:16
   ---------------------------------------- 0.6/53.8 MB 4.4 MB/s eta 0:00:13
    --------------------------------------- 0.8/53.8 MB 4.1 MB/s eta 0:00:13
    --------------------------------------- 0.9/53.8 MB 4.8 MB/s eta 0:00:11
    --------------------------------------- 1.0/53.8 MB 3.9 MB/s eta 0:00:14
    --------------------------------------- 1.0/53.8 MB 3.9 MB/s eta 0:00:14
    --------------------------------------- 1.0/53.8 MB 3.9 MB/s eta 0:00:14
    --------------------------------------- 1.0/53.8 MB 3.9 MB/s eta 0:00:14
    --------------------------------------- 1.0/53.8 MB 3.9 MB/s eta 0:00:14
    --------------------------------------- 1.0/53.8 MB 3.9 MB/s eta 0:00:14
    --------------------------------------- 1.1/53.8 MB 2.0 MB/s eta 0:00:27
   - -

In [2]:
import zipfile
import os
from tableauhyperapi import HyperProcess, Connection, Telemetry, TableName, Inserter, HyperException
import pandas as pd

In [2]:
# Ruta al archivo .twbx
twbx_file_path = 'Atenciones_HisMinsa.twbx'
extracted_folder = 'folder_extraido'
output_csv_path = 'tengohambre.csv'

In [3]:
# Descomprimir el archivo .twbx
with zipfile.ZipFile(twbx_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

In [4]:
# Buscar archivos .hyper o .tde dentro de la carpeta extraída
data_file_path = None
for root, dirs, files in os.walk(extracted_folder):
    for file in files:
        if file.endswith('.hyper') or file.endswith('.tde'):
            data_file_path = os.path.join(root, file)
            break

print(data_file_path)

folder_extraido\Data\TableauTemp\TEMP_0ifl42b0fpp8d811cnnoi11ygtcj.hyper


In [3]:
# Conectar al archivo .hyper y listar las tablas
with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
    with Connection(endpoint=hyper.endpoint, database=data_file_path) as connection:
        # Listar las tablas disponibles en el archivo .hyper
        catalog = connection.catalog
        schema_names = catalog.get_schema_names()
        
        for schema in schema_names:
            table_names = catalog.get_table_names(schema=schema)
            print(f"Tablas en el esquema '{schema}': {table_names}")

        # Acceder a la tabla identificada
        table_name = TableName('Extract', 'Extract')  # Usar el esquema y nombre de tabla correctos
        if table_name in table_names:
            query = f"SELECT * FROM {table_name}"

            # Crear un DataFrame vacío con los nombres de las columnas
            columns = [col.name for col in catalog.get_table_definition(table_name).columns]
            df = pd.DataFrame(columns=columns)
            df_list = []

            try:
                # Iterar sobre las filas del resultado y agregarlas al DataFrame
                with connection.execute_query(query=query) as result:
                    for row in result:
                        row_data = {columns[i]: row[i] for i in range(len(columns))}
                        print(row_data)  # Imprimir cada fila a medida que se obtiene
                        df_list.append(row_data)
            except HyperException as e:
                print(f"Error al ejecutar la consulta: {e}")

            # Convertir la lista de filas a un DataFrame y concatenar
            df = pd.concat([df, pd.DataFrame(df_list)], ignore_index=True)

            # Guardar el DataFrame en un archivo CSV
            df.to_csv(output_csv_path, index=False)
            print(f"Datos guardados en {output_csv_path}")
        else:
            print(f"La tabla '{table_name}' no existe en el archivo .hyper")

NameError: name 'data_file_path' is not defined